In [32]:
# import requests


# x = 0
# token = '5557434028:AAGNGR3Mlv7oWS-2TO3nUIwYysplhE34Wso'
# method = 'sendMessage'
# myuserid = -1001753354502
# text = 'Hello World'
# response = requests.post(
#     url='https://api.telegram.org/bot{0}/{1}'.format(token, method),
#     data={'chat_id': myuserid, 'text': text }
# ).json()
# print(response["result"]["message_id"])
# message_id = response["result"]["message_id"]

# token = '5557434028:AAGNGR3Mlv7oWS-2TO3nUIwYysplhE34Wso'
# method = 'editMessageText'
# myuserid = -1001753354502
# text = 'Hello World 2'
# response = requests.post(
#     url='https://api.telegram.org/bot{0}/{1}'.format(token, method),
#     data={'chat_id': myuserid, 'message_id':message_id,  'text': text }
# ).json()
# print(response)

In [33]:
# # from tqdm.autonotebook import tqdm
# import time
# # for i in tqdm(range(20), desc = 'tqdm() Progress Bar'):
# #     time.sleep(0.5)

# token = '5557434028:AAGNGR3Mlv7oWS-2TO3nUIwYysplhE34Wso'
# chat_id = -1001753354502
# from tqdm.contrib.telegram import tqdm, trange
# # for i in trange(10,desc="downloading"+str(i), token=token, chat_id=chat_id):

# #     time.sleep(2)


# mylistw = open("input.txt").read().splitlines()
# t = tqdm(mylistw, token=token, chat_id=str(chat_id), ascii="=>", unit="files")
# for i in t:
#     t.set_description("Downloading "+str(i)+"\n")
#     time.sleep(2)



Install required packages

In [34]:
%pip install git+https://github.com/openai/whisper.git requests pysrt

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-23y3mkp1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-23y3mkp1
  Resolved https://github.com/openai/whisper.git to commit 7f3e408e092e73d472036ae2e3fba1e7c68ca4e6
  Preparing metadata (setup.py) ... done

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [35]:
import subprocess
from tqdm.contrib.telegram import tqdm, trange
import requests

token = '5557434028:AAGNGR3Mlv7oWS-2TO3nUIwYysplhE34Wso'
method = 'sendMessage'
id = '-1001753354502'
after_22 = False
resume = False


def send_message(text):
    response = requests.post(
        url='https://api.telegram.org/bot{0}/{1}'.format(token, method),
        data={'chat_id': id, 'text': text }
    ).json()
    return response["result"]["message_id"]

mylistw = open("input.txt").read().splitlines()

send_message('Howdy, a new run is detected\nthe first and last files are ' + mylistw[0] + ' and ' + mylistw[-1] + '\n\nbelow is the progress bars of pulling the files and posting the results\n\nfingers crossed 🤞\n\n\nP.S. Also there will be complete list of files processed below.')

for i in tqdm(mylistw, token=token, chat_id=str(id), desc="downloading", unit="files"):
    subprocess.run(
        ["wget", "https://huexrb.ngrok.io/2021-selected/wendy/" + i + ".mp3", "-P", "./input/"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.STDOUT,
        check=True
    )
message_id = send_message('All files are pulled, now starting the transcription')
!touch output.txt


downloading: 100%|██████████| 2/2 [00:03<00:00,  1.86s/files]


In [36]:
import subprocess
import whisper
import pysrt
from datetime import timedelta
from tqdm.contrib.telegram import tqdm, trange
import string
import requests
import os


def get_seconds(time_str):
    hh, mm, ss = time_str.split(":")
    return float(hh) * 3600 + float(mm) * 60 + float(ss)


def get_end_time(line_num):
    return str(get_seconds(str(subs[line_num].end.to_time())))


def get_start_time(line_num):
    return str(get_seconds(str(subs[line_num].start.to_time())))


def get_text(line_num):
    x = subs[line_num].text
    x = x.translate(str.maketrans("", "", string.punctuation))
    return x.upper()


def whole_line(line_num):
    return (
        get_start_time(line_num)
        + " "
        + get_end_time(line_num)
        + " "
        + get_text(line_num)
    )

mylistw = open("input.txt").read().splitlines()
zip_name = mylistw[0] + "-to-" + mylistw[-1] + ".zip"
new_list = list(set(mylistw) - set(os.listdir('./output/')))

if resume==True:
    mylistw = new_list

t = tqdm(mylistw, token=token, chat_id=str(id), desc="processing", unit="files")
for i in t:
    t.set_description("processing "+str(i)+"\n")
    subprocess.run(
        ["whisper", "./input/" + i + ".mp3", "--language", "English", "--model", "tiny", "-o", "output"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.STDOUT,
        check=True
    )
    subs = pysrt.open("./output/" + i + ".mp3.srt")
    data = ""
    length = len(subs)
    for z in range(length):
        data = data + whole_line(z) + "\n"
    with open("./output.txt", "a") as f:
        f.write(i)
        f.write("\n")
        f.write(data)
        f.write("\n")
        f.write("\n")
        

    if after_22==True:
        data = "2021-selected/wendy/" + i + "-wendy2.txt\n" + data
    else:
        data = "2021-selected/wendy/" + i + "-wendy.txt\n" + data

    response = requests.post("https://huexrb.ngrok.io/", data=data)
    if response.status_code == 200:
        send_message(i)

processing 2022-09-06_12-05-53/2 [00:00<?, ?files/s]
processing 2022-09-06_12-05-53?, ?files/s]
:   0%|          | 0/2 [00:45<?, ?files/s]


UnicodeEncodeError: 'latin-1' codec can't encode character '\ufffd' in position 252: Body ('�') is not valid Latin-1. Use body.encode('utf-8') if you want to send it encoded in UTF-8.

In [ ]:
!zip -r $zip_name output output.txt

token = '5557434028:AAGNGR3Mlv7oWS-2TO3nUIwYysplhE34Wso'
method = 'sendDocument'
id = '-1001753354502'
def send_file(text):
    response = requests.post(
        url='https://api.telegram.org/bot{0}/{1}'.format(token, method),
        data={'chat_id': id, 'caption': text},
        files={'document': doc}
    ).json()
    return response

doc = open(zip_name, 'rb')
send_file("test")

In [ ]:
# mylistw = open("input.txt").read().splitlines()
# print(mylistw)
# print(mylistw[0])
# print(mylistw[-1])
# print(x)

# zip_name = mylistw[0] + "-to-" + mylistw[-1] + ".zip"

